In [ ]:
import json
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# ==========================
# Funciones auxiliares de graficos
# ==========================

def build_scores_df(CH_series, NH_series, RCP_series):
    """
    Construye un DataFrame resumen con CH, NH, RCP y Score_Final.
    Recibe Series indexadas por destino: CH_series, NH_series, RCP_series.
    """
    ch_df = CH_series.to_frame(name="CH")
    nh_df = NH_series.to_frame(name="NH")
    rcp_df = RCP_series.to_frame(name="RCP")

    df_scores = ch_df.join(nh_df, how="inner").join(rcp_df, how="inner")

    # Ajusta si tus ponderaciones globales son otras
    w_ch, w_nh, w_rcp = 0.35, 0.35, 0.30
    df_scores["Score_Final"] = (
        w_ch * df_scores["CH"] +
        w_nh * df_scores["NH"] +
        w_rcp * df_scores["RCP"]
    )
    df_scores = df_scores.reset_index().rename(columns={"index": "destino"})
    return df_scores


def plot_score_final_barh(df_scores):
    """
    Barras horizontales del Score_Final, ordenado de mayor a menor.
    """
    data = df_scores.sort_values("Score_Final", ascending=True)
    plt.figure(figsize=(8, 5))
    plt.barh(data["destino"], data["Score_Final"])
    for i, v in enumerate(data["Score_Final"]):
        plt.text(v + 0.005, i, f"{v:.3f}", va="center")
    plt.xlabel("Score final")
    plt.ylabel("Destino")
    plt.title("Ranking de destinos según Score final")
    plt.tight_layout()
    plt.show()


def plot_scores_heatmap(df_scores):
    """
    Heatmap con CH, NH, RCP y Score_Final por destino.
    """
    cols = ["CH", "NH", "RCP", "Score_Final"]
    data = df_scores.set_index("destino")[cols]
    plt.figure(figsize=(6, 4))
    plt.imshow(data, aspect="auto")
    plt.colorbar(label="Valor normalizado")
    plt.xticks(ticks=range(len(cols)), labels=cols, rotation=45, ha="right")
    plt.yticks(ticks=range(len(data.index)), labels=data.index)
    plt.title("Resumen de indicadores por destino")
    plt.tight_layout()
    plt.show()


def plot_price_category_mix(proportion_by_category):
    """
    Grafico de barras apiladas para la mezcla de categorías de precios.
    """
    proportion_by_category.plot(kind='bar', stacked=True, figsize=(10, 6))
    plt.title("Mix de categorías de precio por destino")
    plt.ylabel("Proporción")
    plt.xlabel("Destino")
    plt.legend(title="Categoría", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()


def plot_ch_components(ch_components):
    """
    Grafico de barras agrupadas para los componentes normalizados de CH.
    """
    ch_components.plot(kind='bar', figsize=(12, 6), width=0.8)
    plt.title("Componentes de la Capacidad Hospitalaria (CH)")
    plt.ylabel("Valor Normalizado")
    plt.xlabel("Destino")
    plt.legend(title="Componentes", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


def plot_critical_services(sc_series):
    """
    Grafico de barras horizontales para la proporción de servicios críticos.
    """
    plt.figure(figsize=(10, 6))
    sc_series.sort_values().plot(kind='barh', color='teal')
    plt.title("Proporción de Alojamientos con Servicios Críticos (SC)")
    plt.xlabel("Proporción")
    plt.ylabel("Destino")
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


def plot_nh_components(nh_components):
    """
    Grafico de barras agrupadas para componentes de NH.
    """
    nh_components.plot(kind='bar', figsize=(12, 6), width=0.8)
    plt.title("Componentes del Nivel de Hospitalidad (NH)")
    plt.ylabel("Puntaje")
    plt.xlabel("Destino")
    plt.legend(title="Indicadores", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
"""
Script de planteamiento metodológico y análisis comparativo de resultados
"""

DATA_DIR = Path.cwd().parent / "datos_booking"

#Lectura de archivos JSON generados con web scraper en Booking.com
json_paths = {
    "destinos_ids": DATA_DIR / "destinos_ids.json",
    "manta": DATA_DIR / "manta_data.json",
    "salinas": DATA_DIR / "salinas_data.json",
    "montañita": DATA_DIR / "montañita_data.json",
    "puerto_lopez": DATA_DIR / "puerto_lópez_data.json",
    "general_villamil": DATA_DIR / "general_villamil_data.json",
    "ayampe": DATA_DIR / "ayampe_data.json",
    "atacames": DATA_DIR / "atacames_data.json"
}

data_raw = {}

for key, file_path in json_paths.items():
    with open(file_path, "r", encoding="utf-8") as f:
        content = json.load(f)
        data_raw[key] = content

In [ ]:
# Creación de df con información básica del alojamiento, servicios críticos y reseñas

records = []

for key, content in data_raw.items():
    if "alojamientos" not in content:
        continue  # destinos_ids.json no tiene alojamientos

    destino = content["destino"]

    for aloj in content["alojamientos"]:
        records.append({
            "destino": destino,
            "title": aloj.get("title"),
            "price": aloj.get("price"),
            "rating": aloj.get("rating"),
            "dist_centro_km": aloj.get("distance"),
            "description": aloj.get("description"),
            "services":aloj.get("services", []),
            "reviews": aloj.get("reviews", []),
            "features": aloj.get("features", [])
        })

df = pd.DataFrame(records)

In [ ]:
df

In [ ]:
# Limpieza de valores de precio, remoción de US$ y reemplazo de comas por puntos
df["price"] = (
    df["price"]
    .str.replace("US$", "", regex=False)
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

# Limpieza de valores de rating, reemplazo de comas por puntos
df["rating"] = (
    df["rating"]
    .replace({"Sin puntuación": np.nan})
    .str.replace(",", ".", regex=False)
    .astype(float)
)

# Limpieza de valores de distancia, extracción de flotantes en km
df["dist_centro_km"] = (
    df["dist_centro_km"]
    .str.extract(r"(\d+,\d+|\d+\.\d+|\d+)")
    [0]
    .str.replace(",", ".", regex=False)
    .astype(float)
)

In [ ]:
# Definición de categorías basadas en precios
q1 = df["price"].quantile(0.33)
q2 = df["price"].quantile(0.66)

def categorize_price(p):
    if p <= q1: return "accesible"
    elif p <= q2: return "estándar"
    else: return "premium"

df["categoria"] = df["price"].apply(categorize_price)

In [ ]:
# Categorización de alojamientos con servicios críticos
def has_pool(services):
    return any("piscina" in s.lower() for s in services)

def has_breakfast(features, services):
    text = (features or "") + " " + " ".join(services)
    return "desayuno" in text.lower()

def has_beachfront(services):
    return any("frente a la playa" in s.lower() for s in services)

df["pool"] = df["services"].apply(has_pool)
df["breakfast"] = df.apply(lambda r: has_breakfast(r["features"], r["services"]), axis=1)
df["beachfront"] = df["services"].apply(has_beachfront)

**Capacidad Hospitalaria (CH)**
Est+a definida por los siguientes criterios:

1.   Total de Alojamientos (TA)
2.   Distribución por categoría (DC)
3.   Proximidad media al centro (PC)
4.   Proporción de alojamientos con servicios criticos (SC)

La ponderación de este criterio se dará de la siguiente manera:

CH = 0.4TA + 0.25DC + 0.25PC + 0.1SC





In [ ]:
#1. Total Alojamientos (TA)
TA = df.groupby('destino')["title"].count().rename("TA")
TA

La DC es una descripción de qué tan equitativa es la distribución de los alojamientos por cada categoría de precios. Un valor cercano a 1 indica una distribución más balanceada, la misma cantidad de alojamientos entre cada categoría.

In [ ]:
#2. Distribución por categoría (DC)
cat_counts = df.groupby(
    ["destino", "categoria"]
    )["title"].count().unstack().fillna(0)
DC = 1 - (
    cat_counts.max(axis=1) - cat_counts.min(axis=1)
    ) / cat_counts.sum(axis=1)
DC.name = "DC"
proportion_by_category = cat_counts.div(cat_counts.sum(axis=1), axis=0)
combined_cats = pd.concat([proportion_by_category, DC], axis=1)
display(combined_cats)

In [ ]:
# 1. Mix de categorías de precio por destino
plot_price_category_mix(proportion_by_category)

Aquí tienes una tabla que muestra la distancia media al centro (`dist_mean`) y la proximidad media al centro (`PC`) para cada destino:

In [ ]:
#3. Proximidad media al centro (PC)
dist_mean = df.groupby("destino")["dist_centro_km"].mean()
PC = 1 - (dist_mean - dist_mean.min()) / (dist_mean.max() - dist_mean.min())
PC.name = "PC"
combined_distances = pd.concat([dist_mean, PC], axis=1)
display(combined_distances)

In [ ]:
#4. Proporción de alojamientos con servicios críticos (SC)
SC = (
    (df.groupby("destino")["pool"].mean() +
     df.groupby("destino")["breakfast"].mean() +
     df.groupby("destino")["beachfront"].mean()) / 3
).rename("SC")
SC

In [ ]:
# 2. Proporción de Servicios Críticos (SC)
# Esto muestra qué destino tiene mayor cobertura de servicios clave (piscina, desayuno, frente a playa)
plot_critical_services(SC)

In [ ]:
# Min-Max Scaling, con rango de 0–1
CH_df = pd.concat([TA, DC, PC, SC], axis=1)
CH_norm = (CH_df - CH_df.min()) / (CH_df.max() - CH_df.min())

CH_df["CH"] = (
    0.40 * CH_norm["TA"] +
    0.25 * CH_norm["DC"] +
    0.25 * CH_norm["PC"] +
    0.1 * CH_norm["SC"]
)
CH_df

In [ ]:
# 3. Componentes de Capacidad Hospitalaria (CH)
# Esto nos deja ver si la capacidad es por 'cantidad' (TA) o 'calidad/ubicación' (PC/SC)
ch_components = CH_norm[["TA", "DC", "PC", "SC"]]
plot_ch_components(ch_components)

**Nivel de Hospitalidad (NH)**
Está definida con base a las ponderaciones de los siguientes criterios:
1.   Rating promedio del destino (RP)
2.   Mediana del rating (MR)
3.   Análisis de sentimiento (HF)
4.   Hospitality Experience Score (HES)

La ponderación de este criterio se dará de la siguiente forma:

NH = 0.25HF + 0.2HES + 0.2MR + 0.35RP

In [ ]:
# Extraer reseñas a un DF
rev_rows = []
for _, row in df.iterrows():
    for r in row["reviews"]:
        text = (r.get("positive_feedback", "") or "") + " " + (r.get("negative_feedback", "") or "")
        rev_rows.append({"destino": row["destino"], "review": text})

reviews = pd.DataFrame(rev_rows)
reviews["review"] = reviews["review"].fillna("")

reviews

In [ ]:
#1. Rating promedio del destino (RP) y 2. Mediana del rating (MR)
RP = df.groupby("destino")["rating"].mean().rename("RP")
MR = df.groupby("destino")["rating"].median().rename("MR")
Rating = pd.concat([RP, MR], axis=1)
Rating

In [ ]:
#3. Análisis de sentimiento (OpenAI API) con GPT-4o-mini
"""
Análisis de sentimiento usando OpenAI API con modelo mejorado
"""

from openai import OpenAI
import numpy as np
import os
import re

# Inicializar cliente de OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def analizar_sentimientos(textos, batch_size=10):
    """
    Realiza análisis de sentimientos en español usando OpenAI API.
    Procesamiento por lotes para optimizar llamadas a la API.
    Retorna un score de 1 a 5 estrellas.
    """
    resultados = []
    
    for i in range(0, len(textos), batch_size):
        batch = textos[i:i + batch_size]
        
        for texto in batch:
            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini",  # Modelo mejorado
                    messages=[
                        {
                            "role": "system",
                            "content": """Eres un sistema experto en análisis de reseñas hoteleras. Evalúa la satisfacción del huésped considerando:
                            - Limpieza y mantenimiento
                            - Calidad del servicio y atención del personal
                            - Comodidad de las instalaciones
                            - Relación calidad-precio
                            - Experiencia general

                            Responde ÚNICAMENTE con un número del 1 al 5:
                            1 = Experiencia muy negativa (pésimo hotel)
                            2 = Experiencia negativa (hotel deficiente)
                            3 = Experiencia neutral (hotel aceptable)
                            4 = Experiencia positiva (buen hotel)
                            5 = Experiencia muy positiva (hotel excelente)

                            IMPORTANTE: Responde solo con el número, sin texto adicional."""
                        },
                        {
                            "role": "user",
                            "content": texto[:500]  # Limitar a 500 caracteres para evitar textos muy largos
                        }
                    ],
                    temperature=0,  # Temperatura 0 para respuestas más deterministas
                    max_tokens=5
                )
                
                # Extraer el score de la respuesta
                score_text = response.choices[0].message.content.strip()
                
                # Intentar extraer el número incluso si hay texto adicional
                numeros = re.findall(r'\b[1-5]\b', score_text)
                
                if numeros:
                    score = float(numeros[0])
                else:
                    # Si no se encuentra un número válido, intentar convertir directamente
                    score = float(score_text)
                
                # Asegurar que está en el rango 1-5
                score = max(1.0, min(5.0, score))
                
                resultados.append({"label": f"{score} stars", "score": score})
                
                if (i + len(resultados)) % 50 == 0:
                    print(f"Procesados {i + len(resultados)} textos...")
                
            except ValueError as e:
                print(f"Error de conversión con respuesta: '{score_text}' - Asignando neutro")
                resultados.append({"label": "3 stars", "score": 3.0})
                
            except Exception as e:
                print(f"Error procesando texto: {e}")
                resultados.append({"label": "3 stars", "score": 3.0})
    
    return resultados

# Ejecutar el análisis con todas las reseñas
print("Iniciando análisis de sentimientos con GPT-4o-mini...")
result_sentiment_analysis = analizar_sentimientos(reviews["review"].tolist())

def label_to_stars(label: str) -> float:
    """
    Convierte etiquetas tipo '4 stars' o '1 star' a float (1.0–5.0).
    """
    try:
        return float(label.split()[0])
    except Exception:
        return 3.0  # neutro si algo raro pasa

# Añadimos la columna HF (score de 1 a 5) a cada reseña
reviews["HF"] = [label_to_stars(r["label"]) for r in result_sentiment_analysis]

print(f"\nAnálisis completado. Distribución de sentimientos:")
print(reviews["HF"].value_counts().sort_index())

# Agregamos HF a nivel destino (criterio para NH)
HF_score = reviews.groupby("destino")["HF"].mean().rename("HF")
print(f"\nPromedio de sentimientos por destino:")
print(HF_score)

In [ ]:
HF_score

In [ ]:
#4. Hospitality Experience Score (HES)
from openai import OpenAI
from sklearn.cluster import KMeans
import numpy as np
import os

# Inicializar cliente de OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def obtener_embedding(texto, model="text-embedding-3-small"):
    """
    Obtiene el embedding de un texto usando OpenAI API.
    """
    try:
        texto = texto.replace("\n", " ")
        response = client.embeddings.create(input=[texto], model=model)
        return response.data[0].embedding
    except Exception as e:
        print(f"Error obteniendo embedding: {e}")
        # Retornar un vector de ceros si hay error (dimensión 1536 para text-embedding-3-small)
        return [0.0] * 1536

def obtener_embeddings_batch(textos, batch_size=100, model="text-embedding-3-small"):
    """
    Obtiene embeddings en lotes para optimizar llamadas a la API.
    """
    embeddings = []
    total_textos = len(textos)
    
    print(f"\n📊 Procesando {total_textos} embeddings...")
    
    for i in range(0, len(textos), batch_size):
        batch = textos[i:i + batch_size]
        batch_clean = [texto.replace("\n", " ") if isinstance(texto, str) else "" for texto in batch]
        
        try:
            response = client.embeddings.create(input=batch_clean, model=model)
            batch_embeddings = [item.embedding for item in response.data]
            embeddings.extend(batch_embeddings)
            
            procesados = min(i + batch_size, len(textos))
            porcentaje = (procesados / total_textos) * 100
            barra = "█" * int(porcentaje / 5) + "░" * (20 - int(porcentaje / 5))
            print(f"[{barra}] {procesados}/{total_textos} ({porcentaje:.1f}%)")
            
        except Exception as e:
            print(f"❌ Error en batch {i}-{i+batch_size}: {e}")
            # En caso de error, añadir vectores de ceros
            embeddings.extend([[0.0] * 1536 for _ in batch])
    
    print("✅ Embeddings completados\n")
    return embeddings

# 2) Calcular embeddings de cada reseña usando OpenAI
print("Calculando embeddings con OpenAI...")
embeddings_list = obtener_embeddings_batch(reviews["review"].tolist())
reviews["embedding"] = embeddings_list

# 3) Preparar matriz de características para clustering
X = np.array(reviews["embedding"].tolist())

# 4) Clustering temático (ajuste de n_clusters)
n_clusters = 8
print(f"🔍 Ejecutando clustering con {n_clusters} clusters...")
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
reviews["cluster"] = kmeans.fit_predict(X)
print("✅ Clustering completado\n")

# 5) Calcular promedio de HF por destino y cluster (sentimiento por tema)
cluster_sent = reviews.groupby(["destino", "cluster"])["HF"].mean()

# 6) Hospitality Experience Score:
#    promedio del sentimiento por cluster para cada destino
HES = cluster_sent.groupby("destino").mean().rename("HES")

In [ ]:
#Nivel de Hospitalidad (NH)
NH_df = pd.concat([RP, MR, HF_score, HES], axis=1).fillna(0)

# Min–max Scaling con rango de 0-1
NH_norm = (NH_df - NH_df.min()) / (NH_df.max() - NH_df.min())

# Fórmula que definiste para NH:
# NH = 0.30 RP + 0.15 MR + 0.30 HF + 0.25 HES
NH_df["NH"] = (
    0.30 * NH_norm["RP"] +
    0.15 * NH_norm["MR"] +
    0.30 * NH_norm["HF"] +
    0.25 * NH_norm["HES"]
)
NH_df

In [ ]:
# 4. Componentes del Nivel de Hospitalidad
nh_components = NH_df[["HF", "HES", "RP", "MR"]]
plot_nh_components(nh_components)


**Relacion Calidad-Precio (RCP)**
Está definida con base a las ponderaciones de los siguientes criterios:
1.   Precio promedio por categoría (PPC)
2.   Precio vs. Rating (PVR)
3.   Dispersión de precios (DP)

La ponderación de este criterio se dará de la siguiente manera:

RCP = 0.4PPC + 0.35PVR + 0.25DP  

In [ ]:
#1. Precio promedio por categoría (PPC)
PPC = df.groupby(["destino", "categoria"])["price"].mean().unstack().fillna(0)
PPC = PPC.mean(axis=1).rename("PPC")

In [ ]:
#2. Precio vs. Rating (PVR)
df["PVR"] = df["rating"] / df["price"]
PVR = df.groupby("destino")["PVR"].mean().rename("PVR")

In [ ]:
#3. Dispersión de precios (DP)
DP = df.groupby("destino")["price"].std().fillna(0).rename("DP")

In [ ]:
combined_quality_price = pd.concat([PPC, PVR, DP], axis=1)
display(combined_quality_price)

### Explicación y mejora de la presentación de los resultados:

Aquí se muestra una tabla combinada de los indicadores relacionados con la calidad-precio para cada destino:

**1. Precio promedio por categoría (PPC)**
*   **Descripción:** Este métrica representa el precio promedio de los alojamientos en cada destino, considerando la distribución de precios por categoría (accesible, estándar, premium). Un valor menor indica que, en promedio, los alojamientos en el destino tienen precios más bajos.
*   **Indicaciones:** Es útil para entender el nivel de costo general de un destino. Por ejemplo, un `PPC` alto en Salinas podría indicar que es un destino con alojamientos más caros en general.

**2. Precio vs. Rating (PVR)**
*   **Descripción Faltante:** El `PVR` mide la relación entre la calificación (rating) de un alojamiento y su precio. Se calcula como la calificación dividida por el precio (`rating / price`).
*   **Indicaciones:** Un valor `PVR` **más alto** sugiere que los alojamientos en ese destino ofrecen una **mejor percepción de valor**, es decir, obtienen calificaciones más altas en relación con su precio. Por el contrario, un `PVR` bajo podría indicar que los precios son altos en comparación con las calificaciones recibidas.
*   **Mejor forma de mostrarlo:** Además de la tabla, un gráfico de barras para cada destino que muestre el `PVR` puede hacer que las comparaciones sean más intuitivas. También se podría considerar un gráfico de dispersión con `PPC` en un eje y `PVR` en otro para ver patrones entre el costo general y la percepción de valor.

**3. Dispersión de precios (DP)**
*   **Descripción Faltante:** La `DP` representa la desviación estándar de los precios de los alojamientos en cada destino. Este valor indica la variabilidad o el rango de precios dentro del destino.
*   **Indicaciones:** Un `DP` **alto** sugiere que hay una **gran variedad de precios** en el destino, lo que implica opciones para diferentes presupuestos. Un `DP` bajo, en cambio, indica que los precios de los alojamientos son más homogéneos y menos variados.
*   **Mejor forma de mostrarlo:** Un gráfico de barras para la `DP` podría visualizar fácilmente qué destinos tienen una mayor o menor diversidad de precios. Combinarlo con el `PPC` en un mismo gráfico (por ejemplo, con un eje secundario) también podría ofrecer una visión completa de la estructura de precios de cada destino.

In [ ]:
# Fórmula RCP
RCP_df = pd.concat([PPC, PVR, DP], axis=1)
RCP_norm = (RCP_df - RCP_df.min()) / (RCP_df.max() - RCP_df.min())

RCP_df["RCP"] = (
    0.40 * RCP_norm["PPC"] +
    0.40 * RCP_norm["PVR"] +
    0.20 * RCP_norm["DP"]
)

In [ ]:
#Definición de ponderaciones y combinaciones para el Score Final

final = pd.concat([CH_df["CH"], NH_df["NH"], RCP_df["RCP"]], axis=1)

final["Score_Final"] = (
    0.4 * final["CH"] +
    0.3 * final["NH"] +
    0.2 * final["RCP"]
)

ranking = final.sort_values("Score_Final", ascending=False)
ranking

In [ ]:
# 5 y 6. Resultados finales

# Preparación del dataframe para las funciones de gráfico (necesitan 'destino' como columna)
df_scores_viz = ranking.reset_index()

# 5. Gráfico de barras con el ranking final
plot_score_final_barh(df_scores_viz)

# 6. Heatmap con el detalle de los componentes (CH, NH, RCP)
plot_scores_heatmap(df_scores_viz)

In [ ]:
import os

# 1. Crear carpeta para los outputs
output_dir = Path.cwd().parent / "output"
os.makedirs(output_dir, exist_ok=True)

# 2. Guardar Tablas (DataFrames)
# CSV para uso de datos
ranking.to_csv(f"{output_dir}/ranking_final.csv")
CH_df.to_csv(f"{output_dir}/detalles_CH.csv")
NH_df.to_csv(f"{output_dir}/detalles_NH.csv")
RCP_df.to_csv(f"{output_dir}/detalles_RCP.csv")

# Markdown para visualizar directamente en GitHub (README.md)
with open(f"{output_dir}/ranking_summary.md", "w") as f:
    f.write("# Ranking de Destinos\n\n")
    ranking.to_markdown(f)

# 3. Guardar Gráficos
# Regeneramos los gráficos para guardarlos (plt.show() anterior limpia la figura)

def save_plot(filename):
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{filename}", dpi=300, bbox_inches='tight')
    plt.close()

# Gráfico 1: Ranking Final
df_scores_viz = ranking.reset_index()
data = df_scores_viz.sort_values("Score_Final", ascending=True)
plt.figure(figsize=(12, 5))
plt.barh(data["destino"], data["Score_Final"])
for i, v in enumerate(data["Score_Final"]):
    plt.text(v + 0.005, i, f"{v:.3f}", va="center")
plt.xlabel("Score final")
plt.title("Ranking de destinos según Score final")
save_plot("ranking_final_barh.png")

# Gráfico 2: Heatmap
cols = ["CH", "NH", "RCP", "Score_Final"]
data = df_scores_viz.set_index("destino")[cols]
plt.figure(figsize=(6, 4))
plt.imshow(data, aspect="auto")
plt.colorbar(label="Valor normalizado")
plt.xticks(ticks=range(len(cols)), labels=cols, rotation=45, ha="right")
plt.yticks(ticks=range(len(data.index)), labels=data.index)
plt.title("Resumen de indicadores por destino")
save_plot("heatmap_indicadores.png")

# Gráfico 3: Mix Categorías
proportion_by_category.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title("Mix de categorías de precio por destino")
plt.legend(title="Categoría", bbox_to_anchor=(1.05, 1), loc='upper left')
save_plot("mix_categorias_precios.png")

# Gráfico 4: Componentes CH
CH_norm[["TA", "DC", "PC", "SC"]].plot(kind='bar', figsize=(12, 6), width=0.8)
plt.title("Componentes de la Capacidad Hospitalaria (CH)")
plt.legend(title="Componentes", bbox_to_anchor=(1.05, 1), loc='upper left')
save_plot("componentes_CH.png")

# Gráfico 5: Servicios Críticos
plt.figure(figsize=(10, 6))
SC.sort_values().plot(kind='barh', color='teal')
plt.title("Proporción de Alojamientos con Servicios Críticos (SC)")
save_plot("servicios_criticos.png")

# Gráfico 6: Componentes NH
NH_df[["HF", "HES", "RP", "MR"]].plot(kind='bar', figsize=(12, 6), width=0.8)
plt.title("Componentes del Nivel de Hospitalidad (NH)")
plt.legend(title="Indicadores", bbox_to_anchor=(1.05, 1), loc='upper left')
save_plot("componentes_NH.png")

print(f"Todos los archivos han sido generados exitosamente en la carpeta: {output_dir}/")

In [ ]:
import shutil
from google.colab import files

# Comprimir la carpeta
shutil.make_archive("resultados_analisis", 'zip', "output_github")

# Descargar el archivo zip
files.download("resultados_analisis.zip")